In [1]:
import glob
import itertools
import json
import os
import pandas

pandas.options.display.max_rows = None

In [2]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return int(fragments[2])

In [9]:
# path = "ExperimentData//"
# path = "SampleSessionLayerableOnly//"
# path = "1652473795203//"
# path = "1652484257598//"
# path = "31/PersonalAR_2//"

session_paths = glob.glob("31/PersonalAR*//*")
print(session_paths)
dir_list = []
for session_path in session_paths:
    dir_list.extend(os.listdir(session_path))

dir_list.sort(key=parseFileNumber)

print("Using session paths: ")
print(session_paths)
# print("Files and directories in '", path, "' :")
 
# prints all files
print ("Found following session files: ")
print(dir_list)
# print(dir_list)


['31\\PersonalAR\\']


IndexError: list index out of range

In [4]:
appEvents = []
gestureEvents = []
codesEvents = []
experimentEvents = []

def parseData(json_data):
    frames_list = []
    frames = json_data["frames"]
    for frame in frames:
        frameData = {
            "frameNum": int,
            "timestamp": float,
            "systemTime": str,
            "unixTime": float,
            "gazeValid": bool,

            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},

            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},

            "hand":{},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "openApps": [],
            "appEvents": [],
            "placedObjects": [],
            "anchorEvents":[],
            "codesEvents":[],
            "gestureEvents":[],
            "IsLayerableMode": bool,
            "experimentEvents": []
        }

        appEvent = frame["appEvents"]
        codeEvent = frame["codesEvents"]
        experimentEvent = frame["experimentEvents"]
        gestureEvent = frame["gestureEvents"]

        frameData["frameNum"] = frame["frameNum"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["systemTime"] = frame["systemTime"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["gazeValid"] = frame["gazeValid"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]

        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]

        frameData["hand"] = frame["hand"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["openApps"] = frame["openApps"]
        frameData["appEvents"] = appEvent
        frameData["placedObjects"] = frame["placedObjects"]
        frameData["anchorEvents"] = frame["anchorEvents"]
        frameData["codesEvents"] = codeEvent
        frameData["gestureEvents"] = gestureEvent
        frameData["IsLayerableMode"] = frame["IsLayerableMode"]
        frameData["experimentEvents"] = experimentEvent
        frames_list.append(frameData)
        if appEvent:
            appEvents.append(appEvent)
        if codeEvent:
            codesEvents.append(codeEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
        if gestureEvent:
            gestureEvents.append(gestureEvent)
    return frames_list


In [5]:
frames = []

def extract_frames(session_path):
    session_dir_list = os.listdir(session_path)
    extracted_frames = []
    for filename in session_dir_list:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            #print(filename)
            firstFileFlag = False
            if "Session_0" in filename:
                firstFileFlag = True
            with open(os.path.join(session_path, filename), 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    #data.append(json_data["obj"]["sessionRecording"])
                    session_frames = parseData(json_data["obj"]["sessionRecording"])
                    extracted_frames.append(session_frames)
                else:
                    #data.append(json.load(f))
                    session_frames = parseData(json_data)
                    extracted_frames.append(session_frames)
    return extracted_frames

for session_path in session_paths:
    frames.extend(extract_frames(session_path))

In [7]:
frames
framesUnpacked = list(itertools.chain(*frames))
framesDf = pandas.DataFrame(framesUnpacked)

framesDf = framesDf.sort_values(by='timestamp')
framesDf.head(1)
framesDf.head(2)

KeyError: 'timestamp'

In [ ]:
layerableOpenApps = framesDf.loc[(framesDf['IsLayerableMode'] == True)]
immersiveOpenApps = framesDf.loc[(framesDf['IsLayerableMode'] == False)]

immersiveOpenAppsDf = pandas.DataFrame(immersiveOpenApps["openApps"])
layerableOpenAppsDf = pandas.DataFrame(layerableOpenApps["openApps"])

immersiveOpenAppsDf.head(2)

for im in immersiveOpenAppsDf["openApps"]:
    if len(im) > 1:
        print(im)

avg = 0
count = 0
max = 0
min = 6
for lay in layerableOpenAppsDf["openApps"]:
    numAppOpen = len(lay)
    avg = numAppOpen + avg
    count = count + 1
    if(numAppOpen > max):
        max = numAppOpen
    if(numAppOpen < min):
        min = numAppOpen
    

print(avg/count)
print(max)
print(min)

['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
['Code Entry', 'AnnotationsMenu']
3.528321424873149
5
0


In [ ]:
gestureEvents
gestureEventsUnpacked = list(itertools.chain(*gestureEvents))
gestureEventsDf = pandas.DataFrame(gestureEventsUnpacked)

selectGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Select"]
manipulateGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Manipulate Action"]
navigateGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Navigation Action"]
holdGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Hold Action"]

In [ ]:
leftRay = framesDf.loc[framesDf["leftHandRay"] != {'x': 0.0, 'y': 0.0, 'z': 0.0}]
rightRay = framesDf.loc[framesDf["rightHandRay"] != {'x': 0.0, 'y': 0.0, 'z': 0.0}]

In [ ]:
#experimentEvents[0][0]
experimentEvents
pinCodes = experimentEvents[0][0]["pinCodes"]

for exper in experimentEvents:
    for exp in exper:
        if exp["eventName"] == "EndTask":
            print(exp)


{'unixTime': 1652723595465, 'systemTime': '10-53-15-46', 'userID': '1652722541690', 'sessionType': 'Mode B', 'eventName': 'EndTask', 'pinRandomSeed': 4402, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 4}
{'unixTime': 1652724419089, 'systemTime': '11-06-59-08', 'userID': '1652723950882', 'sessionType': 'Mode A', 'eventName': 'EndTask', 'pinRandomSeed': 6123, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 3}
{'unixTime': 1652724981387, 'systemTime': '11-16-21-38', 'userID': '1652723950882', 'sessionType': 'Mode B', 'eventName': 'EndTask', 'pinRandomSeed': 6123, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 7}
{'unixTime': 1652725457274, 'systemTime': '11-24-17-27', 'userID': '1652723950882', 'sessionType': 'Mode A', 'eventName': 'EndTask', 'pinRandomSeed': 6123, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 12}


In [ ]:
experimentEvents
experimentsUnpacked = list(itertools.chain(*experimentEvents))

experimentsDf = pandas.DataFrame(experimentsUnpacked).sort_values(by="unixTime")
experimentsDf



,unixTime,systemTime,userID,sessionType,eventName,pinRandomSeed,pinNumCodes,pinCodeLength,pinNumPieces,pinCodes,score
0,1652723175440,10-46-15-43,1652722541690,Mode B,StartTask,4402,200,6,3,"[{'Value': 498228, 'Label': 'ND', 'Pieces': [{...",0
1,1652723595465,10-53-15-46,1652722541690,Mode B,EndTask,4402,200,6,3,[],4
4,1652723999053,10-59-59-05,1652723950882,Mode A,StartTask,6123,200,6,3,"[{'Value': 582797, 'Label': 'SD', 'Pieces': [{...",0
2,1652724419089,11-06-59-08,1652723950882,Mode A,EndTask,6123,200,6,3,[],3
3,1652724561361,11-09-21-36,1652723950882,Mode B,StartTask,6123,200,6,3,"[{'Value': 582797, 'Label': 'SD', 'Pieces': [{...",0
5,1652724981387,11-16-21-38,1652723950882,Mode B,EndTask,6123,200,6,3,[],7
6,1652725037268,11-17-17-26,1652723950882,Mode A,StartTask,6123,200,6,3,"[{'Value': 582797, 'Label': 'SD', 'Pieces': [{...",0
7,1652725457274,11-24-17-27,1652723950882,Mode A,EndTask,6123,200,6,3,[],12


In [ ]:
#check experiment events
immersiveStarts = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode B') & (experimentsDf['eventName'] == 'StartTask')]
immersiveEnds = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode B') & (experimentsDf['eventName'] == 'EndTask')]
immersiveStarts.reset_index(drop=True, inplace=True)
immersiveEnds.reset_index(drop=True, inplace=True)
immersiveEnds = immersiveEnds.assign(timeDifference = immersiveEnds['unixTime'].sub(immersiveStarts['unixTime'], fill_value=0))
immersiveTime = immersiveEnds['timeDifference'].sum() / 1000 / 60

layerableStarts = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode A') & (experimentsDf['eventName'] == 'StartTask')]
layerableEnds = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode A') & (experimentsDf['eventName'] == 'EndTask')]
layerableStarts.reset_index(drop=True, inplace=True)
layerableEnds.reset_index(drop=True, inplace=True)
layerableEnds = layerableEnds.assign(timeDifference = layerableEnds['unixTime'].sub(layerableStarts['unixTime'], fill_value=0))
layerableTime = layerableEnds['timeDifference'].sum() / 1000 / 60

experimentStarts = experimentsDf.loc[experimentsDf['eventName'] == 'StartTask']
experimentStops = experimentsDf.loc[experimentsDf['eventName'] == 'EndTask']

immersiveScore = experimentStops.loc[experimentStops['sessionType'] == 'Mode B']['score'].to_numpy()
layerableScore = experimentStops.loc[experimentStops['sessionType'] == 'Mode A']['score'].to_numpy()

In [ ]:
immersiveStartTimes = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode B') & (experimentsDf['eventName'] == 'StartTask')]

In [ ]:
codesEvents
codesUnpacked = list(itertools.chain(*codesEvents))

codesDf = pandas.DataFrame(codesUnpacked)
codesDf.sort_values(by="unixTime")

,unixTime,systemTime,eventTime,codes,success
0,1652722907176,10-41-47-17,365486,511616,False
1,1652722923826,10-42-03-82,382136,165119,False
2,1652722938516,10-42-18-51,396826,161951,False
3,1652722971857,10-42-51-85,430167,165116,False
4,1652723009128,10-43-29-12,467438,161614,False
5,1652723027118,10-43-47-11,485428,141616,False
6,1652723126279,10-45-26-27,584589,145014,True
7,1652723347272,10-49-07-27,805582,261351,True
8,1652723429854,10-50-29-85,888164,326448,True
9,1652723496875,10-51-36-87,955185,871226,True


In [ ]:
successfulCodes = codesDf.loc[codesDf['success'] == True]
successfulCodes.sort_values(by="unixTime")

,unixTime,systemTime,eventTime,codes,success
6,1652723126279,10-45-26-27,584589,145014,True
7,1652723347272,10-49-07-27,805582,261351,True
8,1652723429854,10-50-29-85,888164,326448,True
9,1652723496875,10-51-36-87,955185,871226,True
10,1652723558825,10-52-38-82,1017135,741840,True
31,1652724083565,11-01-23-56,132683,164939,True
35,1652724134507,11-02-14-50,183625,421467,True
36,1652724283737,11-04-43-73,332855,957868,True
13,1652724631882,11-10-31-88,681000,279758,True
14,1652724691203,11-11-31-20,740321,368075,True


In [ ]:
appEvents
appsUnpacked = list(itertools.chain(*appEvents))
appsDf = pandas.DataFrame(appsUnpacked)
appsDf = appsDf.sort_values(by='eventTime')
#appsDf

In [ ]:
#check time spent in each app

tutorialEvents = appsDf[appsDf['name'] == 'Tutorial']
annotationEvents = appsDf[appsDf['name'] == 'AnnotationsMenu']
codeEntryEvents = appsDf[appsDf['name'] == 'Code Entry']
itemInspectorEvents = appsDf[appsDf['name'] == 'Item Inspector']
deviceGroupsEvents = appsDf[appsDf['name'] == 'Device Groups']
atlasEvents = appsDf[appsDf['name'] == 'Atlas']

annotationStarts = annotationEvents[annotationEvents['activity'] == 'Start']
annotationStops = annotationEvents[annotationEvents['activity'] == 'Stop']
annotationStarts.reset_index(drop=True, inplace=True)
annotationStops.reset_index(drop=True, inplace=True)
annotationStops = annotationStops.assign(timeDifference = annotationStops['unixTime'].sub(annotationStarts['unixTime'], fill_value=0))
annotationTime = annotationStops['timeDifference'].sum() / 1000 / 60
print("time spent in annotation menu: " + str(annotationTime))

tutorialStarts = tutorialEvents[tutorialEvents['activity'] == 'Start']
tutorialStops = tutorialEvents[tutorialEvents['activity'] == 'Stop']
tutorialStops.reset_index(drop=True, inplace = True)
tutorialStarts.reset_index(drop = True, inplace = True)
tutorialStops = tutorialStops.assign(timeDifference = tutorialStops['unixTime'].sub(tutorialStarts['unixTime'],fill_value=0))
tutorialTime = tutorialStops['timeDifference'].sum() / 1000 / 60
print("time spent in tutorial: " + str(tutorialTime))

codeEntryStarts = codeEntryEvents[codeEntryEvents['activity'] == 'Start']
codeEntryStops = codeEntryEvents[codeEntryEvents['activity'] == 'Stop']
codeEntryStarts.reset_index(drop=True, inplace=True)
codeEntryStops.reset_index(drop=True, inplace=True)
codeEntryStops = codeEntryStops.assign(timeDifference = codeEntryStops['unixTime'].sub(codeEntryStarts['unixTime'], fill_value=0))
codeEntryTime = codeEntryStops['timeDifference'].sum() / 1000 / 60
print("time spent in code entry: " + str(codeEntryTime))

itemInspectorStarts = itemInspectorEvents[itemInspectorEvents['activity'] == 'Start']
itemInspectorStops = itemInspectorEvents[itemInspectorEvents['activity'] == 'Stop']
itemInspectorStarts.reset_index(drop=True, inplace=True)
itemInspectorStops.reset_index(drop=True, inplace=True)
itemInspectorStops = itemInspectorStops.assign(timeDifference = itemInspectorStops['unixTime'].sub(itemInspectorStarts['unixTime'], fill_value=0))
itemInspectorTime = itemInspectorStops['timeDifference'].sum() / 1000 / 60
print("time spent in item inspector: " + str(itemInspectorTime))

deviceGroupsStarts = deviceGroupsEvents[deviceGroupsEvents['activity'] == 'Start']
deviceGroupsStops = deviceGroupsEvents[deviceGroupsEvents['activity'] == 'Stop']
deviceGroupsStarts.reset_index(drop=True, inplace=True)
deviceGroupsStops.reset_index(drop=True, inplace=True)
deviceGroupsStops = deviceGroupsStops.assign(timeDifference = deviceGroupsStops['unixTime'].sub(deviceGroupsStarts['unixTime'], fill_value = 0))
deviceGroupsTime = deviceGroupsStops['timeDifference'].sum() / 1000 / 60
print("time spent in device groups: " + str(deviceGroupsTime))

atlasStarts = atlasEvents[atlasEvents['activity'] == 'Start']
atlasStops = atlasEvents[atlasEvents['activity'] == 'Stop']
atlasStarts.reset_index(drop=True, inplace=True)
atlasStops.reset_index(drop=True, inplace=True)
atlasStops = atlasStops.assign(timeDifference = atlasStops['unixTime'].sub(atlasStarts['unixTime'], fill_value=0))
atlasTime = atlasStops['timeDifference'].sum() / 1000 / 60
print("time spent in atlas: " + str(atlasTime))

print("\ntime spent in immersive mode: " + str(immersiveTime))
print("time spent in layerable mode: " + str(layerableTime))




time spent in annotation menu: 0.0845
time spent in tutorial: 8.300466666666667
time spent in code entry: 19.219533333333334
time spent in item inspector: 19.505283333333335
time spent in device groups: 18.347550000000002
time spent in atlas: 18.3929

time spent in immersive mode: 14.000850000000002
time spent in layerable mode: 14.0007


In [ ]:

print("number of experiment starts: " + str(experimentStarts.shape[0]))
print("number of experiment stops: " + str(experimentStops.shape[0]))

number of experiment starts: 4
number of experiment stops: 4


In [ ]:
print("number of correct codes entered: " + str(successfulCodes.shape[0]))
for score in immersiveScore:
    print("number of correct codes entered in immersive mode: " + str(score))
for score in layerableScore:
    print("number of correct codes entered in layerable mode: " + str(score))

number of correct codes entered: 26
number of correct codes entered in immersive mode: 4
number of correct codes entered in immersive mode: 7
number of correct codes entered in layerable mode: 3
number of correct codes entered in layerable mode: 12


In [ ]:
placedObject = framesDf.loc[:,"placedObjects"]

for p in placedObject:
    if( len(p) < 15):
        print(len(p))

In [ ]:
print("number of times left ray was tracked: " + str(leftRay.shape[0]))
print("number of times right ray was tracked: " + str(rightRay.shape[0]))

print("\nnumber of select gestures: " + str(selectGesture.shape[0]))
print("number of manipulate gestures: " + str(manipulateGesture.shape[0]))
print("number of navigate gestures: " + str(navigateGesture.shape[0]))
print("number of hold gestures: " + str(holdGesture.shape[0]))

number of times left ray was tracked: 389
number of times right ray was tracked: 5617

number of select gestures: 2
number of manipulate gestures: 136
number of navigate gestures: 135
number of hold gestures: 10
